<a href="https://colab.research.google.com/github/mgrzegorz/first/blob/master/kaggle_project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Mounting Google Drive

In [14]:
!pip install featuretools

In [0]:
import featuretools as ft

In [0]:
!pip install sklearn

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import KFold
import time
import sys
import datetime

import lightgbm as lgb
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', 500)

In [0]:
# zmiana working directory
os.chdir("/content/gdrive/My Drive/Data/Elo/")

In [0]:
#source: https://stackoverflow.com/questions/48750199/google-colaboratory-misleading-information-about-its-gpu-only-5-ram-available
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
#XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
  process = psutil.Process(os.getpid())
  print("Gen RAM Free: " + humanize.naturalsize(psutil.virtual_memory().available), " | Proc size: " + humanize.naturalsize(process.memory_info().rss))
  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

  Running setup.py bdist_wheel for gputil ... - done
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 12.9 GB  | Proc size: 141.9 MB
GPU RAM Free: 11441MB | Used: 0MB | Util   0% | Total 11441MB


In [0]:
#%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [0]:
!pip install -U -q kaggle
!mkdir -p ~/.kaggle

In [0]:
from google.colab import files
files.upload()

In [0]:
!cp kaggle.json ~/.kaggle/

In [0]:
!chmod 600 /root/.kaggle/kaggle.json

In [0]:
!kaggle competitions download -c elo-merchant-category-recommendation -p /content/gdrive/My\ Drive/Data

/bin/bash: kaggle: command not found


In [0]:
merchants_types = {
  'merchant_group_id': 'uint32',
  'merchant_category_id':'int16',
  'city_id': 'int16',
  'state_id': 'int8',
  'subsector_id': 'int8',
  'numerical_1': 'float16',
  'numerical_2': 'float16',
  'category_1': 'category', #bool_,
  'category_2': 'category',
  'category_4': 'category',   
  'most_recent_sales_range': 'category',
  'most_recent_purchases_range': 'category',
  'avg_sales_lag3': 'float32',
  'avg_purchases_lag3': 'float32',
  'active_months_lag3': 'float16',
  'avg_sales_lag6': 'float32',
  'avg_purchases_lag6': 'float32',
  'active_months_lag6': 'float16',
  'avg_sales_lag12': 'float32',
  'avg_purchases_lag12': 'float32',
  'active_months_lag12': 'float16'
 }
types_new_merchant_transactions = {
  'merchant_id':'object',
  'authorized_flag':'category',
  'city_id': 'int16',
  'state_id': 'int8',    
  'subsector_id': 'int8',
  'category_1': 'category',
  'category_2': 'category',
  'category_3': 'category',
  'installments': 'int16',
  'merchant_category_id': 'int16',
  'month_lag': 'int8', #1,2 
  'purchase_amount': 'float32'
  }
types_historical_transactions = {
  'authorized_flag':'category',
  'city_id': 'int16',
  'state_id': 'int8',
  'subsector_id': 'int8',
  'category_1': 'category',
  'category_2': 'category',
  'category_3': 'category',    
  'installments': 'int16',
  'merchant_category_id': 'int16',     
  'merchant_id':'object',
  'month_lag': 'int8',   
  'purchase_amount': 'float32',
}
types_train ={
    'feature_1': 'uint8',
    'feature_2': 'uint8',
    'feature_3': 'bool_',
    'target': 'float16'
}
types_test ={
    'feature_1': 'uint8',
    'feature_2': 'uint8',
    'feature_3': 'bool_'
}

In [0]:
def binarize(df):
    for col in ['authorized_flag', 'category_1']:
        df[col] = df[col].map({'Y':1, 'N':0})
    return df

In [0]:
new_transactions = pd.read_csv("new_merchant_transactions.csv", index_col = 'card_id', parse_dates=['purchase_date'], dtype=types_new_merchant_transactions)
#new_transactions['purchase_date'] = pd.to_datetime(new_merchant_transactions['purchase_date'], unit='ns')
historical_transactions = pd.read_csv("historical_transactions.csv", index_col = 'card_id', parse_dates=['purchase_date'], dtype=types_historical_transactions)
#historical_transactions['purchase_date'] = pd.to_datetime(historical_transactions['purchase_date'], unit='ns')
train = pd.read_csv("train.csv", index_col = 'card_id', parse_dates=['first_active_month'], dtype = types_train)
#train['first_active_month'] = pd.to_datetime(train['first_active_month'])
test = pd.read_csv("test.csv", index_col = 'card_id', parse_dates=['first_active_month'], dtype = types_test)
#test['first_active_month'] = pd.to_datetime(test['first_active_month'])

Featuretools w akcji

In [0]:
merchants.reset_index(inplace = True) #merchant_id
new_transactions.reset_index(inplace = True) #brak własnego id, do łączenia card_id a w drugą stronę merchant_id
historical_transactions.reset_index(inplace = True) #brak własnego id, do łączenia card_id, a w drugą stronę merchant_id
train.reset_index(inplace = True) #card_id

In [0]:
merchants = merchants[~merchants['merchant_id'].duplicated()]

In [0]:
es = ft.EntitySet(id = 'card')
es = es.entity_from_dataframe(entity_id = 'card', 
                              dataframe = train, 
                              index = 'card_id')
es = es.entity_from_dataframe(entity_id = 'new_transactions', 
                              dataframe = new_transactions,
                              make_index = True,
                              index = 'new_transactions_id',
                              time_index = 'purchase_date')
es = es.entity_from_dataframe(entity_id = 'historical_transactions', 
                              dataframe = historical_transactions,
                              make_index = True,
                              index = 'historical_transactions_id',
                              time_index = 'purchase_date')
es = es.entity_from_dataframe(entity_id = 'merchants', 
                              dataframe = merchants, 
                              index = 'merchant_id')

In [1]:
r_card_new_transactions = ft.Relationship(es['card']['card_id'],
                                    es['new_transactions']['card_id'])
r_card_historical_transactions = ft.Relationship(es['card']['card_id'],
                                    es['historical_transactions']['card_id'])
r_new_transactions_merchants = ft.Relationship(es['new_transactions']['merchant_id'],
                                              es['merchants']['merchant_id'])
r_historical_transactions_merchants = ft.Relationship(es['historical_transactions']['merchant_id'],
                                              es['merchants']['merchant_id'])

# Add the relationship to the entity set
es = es.add_relationship(r_client_previous)

NameError: ignored

In [0]:
features, feature_names = ft.dfs(entityset = es, target_entity = 'card', 
                                 agg_primitives = ['mean'],
                                 trans_primitives = ['years', 'month', 'subtract'])

In [20]:
es

Entityset: card
  Entities:
    card [Rows: 201917, Columns: 5]
  Relationships:
    No relationships

In [0]:
target = train['target']
del train['target']

In [0]:
merchants = pd.read_csv("merchants.csv", index_col='merchant_id', dtype = merchants_types)
#merchants = pd.get_dummies(merchants, columns = ['category_1', 'category_2', 'category_4', 'state_id'])
#merchants['most_recent_purchases_range'] = merchants['most_recent_purchases_range'].cat.codes #ordinal
#merchants['most_recent_sales_range'] = merchants['most_recent_sales_range'].cat.codes         #ordinal
merchants.reset_index(inplace=True)

In [0]:
# https://www.kaggle.com/fabiendaniel/elo-world
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [0]:
historical_transactions = binarize(historical_transactions)
new_transactions = binarize(new_transactions)

Feature enginering

In [0]:
historical_transactions['month_diff'] = ((datetime.datetime.today() - historical_transactions['purchase_date']).dt.days)//30
historical_transactions['month_diff'] += historical_transactions['month_lag']

new_transactions['month_diff'] = ((datetime.datetime.today() - new_transactions['purchase_date']).dt.days)//30
new_transactions['month_diff'] += new_transactions['month_lag']

In [9]:
historical_transactions = pd.get_dummies(historical_transactions, columns=['category_2', 'category_3'])
new_transactions = pd.get_dummies(new_transactions, columns=['category_2', 'category_3'])

historical_transactions = reduce_mem_usage(historical_transactions)
new_transactions = reduce_mem_usage(new_transactions)



NameError: ignored

In [0]:
agg_fun = {'authorized_flag': ['mean']}
auth_mean = historical_transactions.groupby(['card_id']).agg(agg_fun)
auth_mean.columns = ['_'.join(col).strip() for col in auth_mean.columns.values]
auth_mean.reset_index(inplace=True)

In [0]:
authorized_transactions = historical_transactions[historical_transactions['authorized_flag'] == 1]
historical_transactions = historical_transactions[historical_transactions['authorized_flag'] == 0]

In [0]:
historical_transactions['purchase_month'] = historical_transactions['purchase_date'].dt.month
authorized_transactions['purchase_month'] = authorized_transactions['purchase_date'].dt.month
new_transactions['purchase_month'] = new_transactions['purchase_date'].dt.month

In [0]:
pd.DatetimeIndex(historical_transactions['purchase_date']).astype(np.int64) * 1e-9

Float64Index([1491569889.0, 1505688027.0, 1505688026.0, 1519124270.0,
              1499499211.0, 1490197988.0, 1499499150.0, 1499499169.0,
              1490197972.0, 1504817839.0,
              ...
              1509063312.0, 1501849863.0, 1501849937.0, 1501849922.0,
              1501849995.0, 1501356749.0, 1507835014.0, 1516012473.0,
              1488660385.0, 1500106669.0],
             dtype='float64', name='purchase_date', length=2516909)

First grouping function

In [0]:
def aggregate_transactions(history):
    
    history.loc[:, 'purchase_date'] = pd.DatetimeIndex(history['purchase_date']).\
                                      astype(np.int64) * 1e-9
    
    agg_func = {
    'category_1': ['sum', 'mean'],
    'category_2_1.00000000': ['mean'],
    'category_2_2.00000000': ['mean'],
    'category_2_3.00000000': ['mean'],
    'category_2_4.00000000': ['mean'],
    'category_2_5.00000000': ['mean'],
    'category_3_A': ['mean'],
    'category_3_B': ['mean'],
    'category_3_C': ['mean'],
    'merchant_id': ['nunique'],
    'merchant_category_id': ['nunique'],
    'state_id': ['nunique'],
    'city_id': ['nunique'],
    'subsector_id': ['nunique'],
    'purchase_amount': ['sum', 'mean', 'max', 'min', 'std'],
    'installments': ['sum', 'mean', 'max', 'min', 'std'],
    'purchase_month': ['mean', 'max', 'min', 'std'],
    'purchase_date': [np.ptp, 'min', 'max'],
    'month_lag': ['mean', 'max', 'min', 'std'],
    'month_diff': ['mean']
    }
    
    agg_history = history.groupby(['card_id']).agg(agg_func)
    agg_history.columns = ['_'.join(col).strip() for col in agg_history.columns.values]
    agg_history.reset_index(inplace=True)
    
    df = (history.groupby('card_id')
          .size()
          .reset_index(name='transactions_count'))
    
    agg_history = pd.merge(df, agg_history, on='card_id', how='left')
    
    return agg_history

In [0]:
history = aggregate_transactions(historical_transactions)
history.columns = ['hist_' + c if c != 'card_id' else c for c in history.columns]
history[:5]

,card_id,hist_transactions_count,hist_category_1_sum,hist_category_1_mean,hist_category_2_1.00000000_mean,hist_category_2_2.00000000_mean,hist_category_2_3.00000000_mean,hist_category_2_4.00000000_mean,hist_category_2_5.00000000_mean,hist_category_3_A_mean,...,hist_purchase_month_min,hist_purchase_month_std,hist_purchase_date_ptp,hist_purchase_date_min,hist_purchase_date_max,hist_month_lag_mean,hist_month_lag_max,hist_month_lag_min,hist_month_lag_std,hist_month_diff_mean
0,C_ID_00007093c1,35,4.0,0.114286,0.000000,0.000000,0.885714,0.0,0.000000,0.000000,...,1,3.071419,0.028858,1.489250,1.518108,-6.028571,0,-11,3.535415,10.028571
1,C_ID_0001238066,3,0.0,0.000000,0.333333,0.000000,0.000000,0.0,0.666667,0.000000,...,2,5.773503,0.003609,1.514660,1.518269,-1.333333,0,-2,1.154701,10.000000
2,C_ID_0001506ef0,4,0.0,0.000000,0.000000,0.000000,1.000000,0.0,0.000000,0.750000,...,2,4.725816,0.022099,1.496772,1.518871,-2.500000,0,-8,3.785939,10.250000
3,C_ID_0001793786,27,2.0,0.074074,0.111111,0.296296,0.111111,0.0,0.000000,0.814815,...,3,2.375684,0.016780,1.488636,1.505416,-3.518519,-1,-7,2.375684,14.185185
4,C_ID_000183fdda,7,0.0,0.000000,0.000000,0.000000,1.000000,0.0,0.000000,0.000000,...,8,0.755929,0.006702,1.502099,1.508801,-5.714286,-4,-6,0.755929,10.000000


In [0]:
authorized = aggregate_transactions(authorized_transactions)
authorized.columns = ['auth_' + c if c != 'card_id' else c for c in authorized.columns]
authorized[:5]

,card_id,auth_transactions_count,auth_category_1_sum,auth_category_1_mean,auth_category_2_1.00000000_mean,auth_category_2_2.00000000_mean,auth_category_2_3.00000000_mean,auth_category_2_4.00000000_mean,auth_category_2_5.00000000_mean,auth_category_3_A_mean,...,auth_purchase_month_min,auth_purchase_month_std,auth_purchase_date_ptp,auth_purchase_date_min,auth_purchase_date_max,auth_month_lag_mean,auth_month_lag_max,auth_month_lag_min,auth_month_lag_std,auth_month_diff_mean
0,C_ID_00007093c1,114,24.0,0.210526,0.000000,0.000000,0.780702,0.0,0.008772,0.0,...,1,3.371490,32627654.0,1.487081e+09,1.519708e+09,-5.798246,0,-12,3.441495,10.315789
1,C_ID_0001238066,120,2.0,0.016667,0.783333,0.000000,0.000000,0.0,0.150000,0.0,...,1,4.895483,13110825.0,1.506638e+09,1.519748e+09,-1.825000,0,-5,1.294218,10.108333
2,C_ID_0001506ef0,62,0.0,0.000000,0.032258,0.000000,0.967742,0.0,0.000000,1.0,...,1,4.538017,34460275.0,1.484411e+09,1.518871e+09,-4.983871,0,-13,4.248402,10.193548
3,C_ID_0001793786,189,0.0,0.000000,0.042328,0.359788,0.063492,0.0,0.000000,1.0,...,1,2.301491,24487497.0,1.484994e+09,1.509481e+09,-3.301587,0,-9,2.301491,14.148148
4,C_ID_000183fdda,137,4.0,0.029197,0.051095,0.007299,0.905109,0.0,0.007299,0.0,...,1,4.538289,15148616.0,1.504444e+09,1.519592e+09,-2.284672,0,-5,1.782055,10.145985


In [0]:
new = aggregate_transactions(new_transactions)
new.columns = ['new_' + c if c != 'card_id' else c for c in new.columns]
new[:5]

,card_id,new_transactions_count,new_category_1_sum,new_category_1_mean,new_category_2_1.00000000_mean,new_category_2_2.00000000_mean,new_category_2_3.00000000_mean,new_category_2_4.00000000_mean,new_category_2_5.00000000_mean,new_category_3_A_mean,...,new_purchase_month_min,new_purchase_month_std,new_purchase_date_ptp,new_purchase_date_min,new_purchase_date_max,new_month_lag_mean,new_month_lag_max,new_month_lag_min,new_month_lag_std,new_month_diff_mean
0,C_ID_00007093c1,2,0,0.000000,0.500000,0.000000,0.50000,0.0,0.000000,0.0,...,4,0.000000,537024.0,1.522754e+09,1.523291e+09,2.000000,2,2,0.000000,10.500000
1,C_ID_0001238066,26,2,0.076923,0.769231,0.000000,0.00000,0.0,0.115385,0.0,...,3,0.485165,5195343.0,1.519923e+09,1.525118e+09,1.346154,2,1,0.485165,10.153846
2,C_ID_0001506ef0,2,0,0.000000,0.000000,0.000000,1.00000,0.0,0.000000,1.0,...,3,0.000000,471152.0,1.521239e+09,1.521710e+09,1.000000,1,1,0.000000,10.000000
3,C_ID_0001793786,31,0,0.000000,0.483871,0.258065,0.16129,0.0,0.032258,1.0,...,11,0.475191,3981096.0,1.510761e+09,1.514742e+09,1.322581,2,1,0.475191,14.000000
4,C_ID_000183fdda,11,0,0.000000,0.000000,0.000000,1.00000,0.0,0.000000,0.0,...,3,0.467099,5106807.0,1.519994e+09,1.525100e+09,1.272727,2,1,0.467099,10.272727


Second grouping function

In [0]:
def aggregate_per_month(history):
    grouped = history.groupby(['card_id', 'month_lag'])

    agg_func = {
            'purchase_amount': ['count', 'sum', 'mean', 'min', 'max', 'std'],
            'installments': ['count', 'sum', 'mean', 'min', 'max', 'std'],
            }

    intermediate_group = grouped.agg(agg_func)
    intermediate_group.columns = ['_'.join(col).strip() for col in intermediate_group.columns.values]
    intermediate_group.reset_index(inplace=True)

    final_group = intermediate_group.groupby('card_id').agg(['mean', 'std'])
    final_group.columns = ['_'.join(col).strip() for col in final_group.columns.values]
    final_group.reset_index(inplace=True)
    
    return final_group
#___________________________________________________________
final_group =  aggregate_per_month(authorized_transactions) 
final_group[:10]

,card_id,month_lag_mean,month_lag_std,purchase_amount_count_mean,purchase_amount_count_std,purchase_amount_sum_mean,purchase_amount_sum_std,purchase_amount_mean_mean,purchase_amount_mean_std,purchase_amount_min_mean,...,installments_sum_mean,installments_sum_std,installments_mean_mean,installments_mean_std,installments_min_mean,installments_min_std,installments_max_mean,installments_max_std,installments_std_mean,installments_std_std
0,C_ID_00007093c1,-6.000000,3.894440,8.769231,3.539158,-4.803313,2.175011,-0.536131,0.075418,-0.702649,...,11.307692,4.210792,1.369646,0.423190,1.000000,0.000000,2.538462,1.613246,0.625428,0.675607
1,C_ID_0001238066,-2.500000,1.870829,20.000000,11.696153,-11.776212,6.676914,-0.603215,0.046045,-0.730351,...,32.500000,24.501020,1.652873,0.410539,0.666667,0.816497,4.666667,2.875181,1.256525,0.521292
2,C_ID_0001506ef0,-6.230769,4.225988,4.769231,4.342692,-2.438161,2.902502,-0.446660,0.271125,-0.725724,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,C_ID_0001793786,-4.500000,3.027650,18.900000,12.160501,-2.496067,4.018810,-0.005687,0.271221,-0.646277,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,C_ID_000183fdda,-2.500000,1.870829,22.833333,7.167054,-11.316092,4.800612,-0.489227,0.141015,-0.728405,...,40.833333,14.048725,1.834943,0.362557,0.333333,1.032796,9.166667,2.041241,2.144389,0.665529
5,C_ID_00024e244b,-6.384615,4.330867,4.076923,4.290717,-2.557395,2.089708,-0.685118,0.071894,-0.728885,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,C_ID_0002709b5a,-5.000000,3.316625,6.090909,3.505839,-3.994622,2.440540,-0.638637,0.060836,-0.700104,...,10.727273,8.427229,1.965729,1.287108,0.818182,0.603023,3.818182,3.341203,1.600058,1.734256
7,C_ID_00027503e2,-4.000000,2.738613,2.666667,1.224745,-1.974976,0.909524,-0.740077,0.002424,-0.742206,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,C_ID_000298032a,-5.250000,4.334249,3.500000,1.772811,-1.844563,0.699127,-0.590214,0.168008,-0.686623,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,C_ID_0002ba3c2e,-4.666667,3.141125,9.166667,4.665476,-5.843051,3.021165,-0.638037,0.044976,-0.721448,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [0]:
def successive_aggregates(df, field1, field2):
    t = df.groupby(['card_id', field1])[field2].mean()
    u = pd.DataFrame(t).reset_index().groupby('card_id')[field2].agg(['mean', 'min', 'max', 'std'])
    u.columns = [field1 + '_' + field2 + '_' + col for col in u.columns.values]
    u.reset_index(inplace=True)
    return u

In [0]:
additional_fields = successive_aggregates(new_transactions, 'category_1', 'purchase_amount')
additional_fields = additional_fields.merge(successive_aggregates(new_transactions, 'installments', 'purchase_amount'),
                                            on = 'card_id', how='left')
additional_fields = additional_fields.merge(successive_aggregates(new_transactions, 'city_id', 'purchase_amount'),
                                            on = 'card_id', how='left')
additional_fields = additional_fields.merge(successive_aggregates(new_transactions, 'category_1', 'installments'),
                                            on = 'card_id', how='left')

In [0]:
history.index = history['card_id']
del history['card_id']

authorized.index = authorized['card_id']
del authorized['card_id']

new.index = new['card_id']
del new['card_id']

final_group.index = final_group['card_id']
del final_group['card_id']

auth_mean.index = auth_mean['card_id']
del auth_mean['card_id']

additional_fields.index = additional_fields['card_id']
del additional_fields['card_id']

In [0]:
train = pd.merge(train, history, left_index = True, right_index = True, how='left')
test = pd.merge(test, history, left_index = True, right_index = True, how='left')

train = pd.merge(train, authorized, left_index = True, right_index = True,  how='left')
test = pd.merge(test, authorized, left_index = True, right_index = True,  how='left')

train = pd.merge(train, new, left_index = True, right_index = True,  how='left')
test = pd.merge(test, new, left_index = True, right_index = True,  how='left')

train = pd.merge(train, final_group, left_index = True, right_index = True,  how='left')
test = pd.merge(test, final_group, left_index = True, right_index = True,  how='left')

train = pd.merge(train, auth_mean, left_index = True, right_index = True,  how='left')
test = pd.merge(test, auth_mean, left_index = True, right_index = True,  how='left')

train = pd.merge(train, additional_fields, left_index = True, right_index = True,  how='left')
test = pd.merge(test, additional_fields, left_index = True, right_index = True,  how='left')

In [0]:
train.shape

(201917, 161)

Training

In [0]:
param = {'num_leaves': 111,
         'min_data_in_leaf': 149, 
         'objective':'regression',
         'max_depth': 9,
         'learning_rate': 0.005,
         "boosting": "gbdt",
         "feature_fraction": 0.7522,
         "bagging_freq": 1,
         "bagging_fraction": 0.7083 ,
         "bagging_seed": 11,
         "metric": 'rmse',
         "lambda_l1": 0.2634,
         "random_state": 133,
         "verbosity": -1}

In [0]:
features = [c for c in train.columns if c not in ['card_id', 'first_active_month']]
#features = [f for f in features if f not in unimportant_features]
categorical_feats = ['feature_2', 'feature_3']

In [0]:
folds = KFold(n_splits=5, shuffle=True, random_state=15)
oof = np.zeros(len(train))
predictions = np.zeros(len(test))

start = time.time()
feature_importance_df = pd.DataFrame()

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train.values, target.values)):
    print("fold n°{}".format(fold_))
    trn_data = lgb.Dataset(train.iloc[trn_idx][features],
                           label=target.iloc[trn_idx],
                           categorical_feature=categorical_feats
                          )
    val_data = lgb.Dataset(train.iloc[val_idx][features],
                           label=target.iloc[val_idx],
                           categorical_feature=categorical_feats
                          )

    num_round = 10000
    clf = lgb.train(param,
                    trn_data,
                    num_round,
                    valid_sets = [trn_data, val_data],
                    verbose_eval=100,
                    early_stopping_rounds = 200)
    
    oof[val_idx] = clf.predict(train.iloc[val_idx][features], num_iteration=clf.best_iteration)
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = features
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    predictions += clf.predict(test[features], num_iteration=clf.best_iteration) / folds.n_splits

print("CV score: {:<8.5f}".format(mean_squared_error(oof, target)**0.5))

fold n°0


/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:1186: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:752: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 200 rounds.
[100]	training's rmse: 3.71482	valid_1's rmse: 3.77687
[200]	training's rmse: 3.6452	valid_1's rmse: 3.72851
[300]	training's rmse: 3.6016	valid_1's rmse: 3.70492
[400]	training's rmse: 3.5706	valid_1's rmse: 3.69259
[500]	training's rmse: 3.54672	valid_1's rmse: 3.68555
[600]	training's rmse: 3.52818	valid_1's rmse: 3.68053
[700]	training's rmse: 3.51298	valid_1's rmse: 3.6779
[800]	training's rmse: 3.49837	valid_1's rmse: 3.67568
[900]	training's rmse: 3.48484	valid_1's rmse: 3.67436
[1000]	training's rmse: 3.47302	valid_1's rmse: 3.67359
[1100]	training's rmse: 3.46096	valid_1's rmse: 3.67261
[1200]	training's rmse: 3.44977	valid_1's rmse: 3.6723
[1300]	training's rmse: 3.43912	valid_1's rmse: 3.6723
[1400]	training's rmse: 3.42847	valid_1's rmse: 3.67176
[1500]	training's rmse: 3.41829	valid_1's rmse: 3.67177
[1600]	training's rmse: 3.40811	valid_1's rmse: 3.6715
[1700]	training's rmse: 3.39807	valid_1's rmse: 3.67133
[

/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:32: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims)


In [0]:
new_merchant_transactions.info()

In [0]:
df = pd.merge(train, historical_transactions, on='card_id', how='left')

In [0]:
merchants['most_recent_purchases_range'].value_counts()

In [0]:
merchants.info()

In [0]:
train.shape